# Урок 15. Консультация по курсовому проекту. Создание чат-бота в Telegram

* Продумать тему курсового проекта.
* Подготовить данные для обучения.
* Продумать интенты на которые вы будете обучаться.

Для сдачи задания необходимо написать:
* На какую тему ваш бот будет общаться (на каких датасетах планируете обучать модели)
* Написать на какие интенты вы будете обучать бота


Примеры интентов
* болталка
* погода в городе
* время в городе
* праздники
* фильмы
* покупка товара
* можно добавить что-то своё

Из обязательных (всего обязательно 3-ри интента) и один из них - болталка

Выбранные интенты:

1. болталка (обязательно)
2. текущее время
3. переводчик с английского (модель, созданная на 10-ом уроке)

In [4]:
# !pip install python-telegram-bot pymorphy2 stop_words annoy gensim catboost --upgrade

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import string
import pickle
import re
import numpy as np
import pandas as pd
from datetime import datetime
import annoy
from multiprocessing import cpu_count
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText
from tqdm.notebook import tqdm as tqdm_notebook
from catboost import CatBoostClassifier
from telegram.ext import Updater, CommandHandler, MessageHandler, CallbackContext, filters as Filters
from telegram import Update
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from chat import preprocess_txt, tqdm_preprocess_txt, check_in_list, embed_txt
from tools import paralell_execution, ar_split_eq_cpu

## Интент болталки

Тренируем и сохраняем для дальнейшего использования

In [2]:
# Подготавливаем ответы и вопросы для обучения
question = None
written = False
with open("prepared_answers.txt", "w") as fout:
    with open("./Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [3]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation) 

In [5]:
file_lines = None
with open("./Otvety.txt", "r") as f:
    file_lines = list(f)[:500000]

args_parallel = []
for lines in ar_split_eq_cpu(file_lines):
    args_parallel.append({
        'lines': lines,
        'sw': sw, 
        'morpher': morpher, 
        'exclude': exclude, 
        'skip_stop_word': True
    })

# запускаем процесс параллельных вычислений
res = paralell_execution(func=tqdm_preprocess_txt,
                         arg=args_parallel,
                         method='multiprocessing')

sentences = [item for row in res for item in row]

In [6]:
# Сохраняем результаты
with open('./sentences.pkl', 'wb') as f:
   pickle.dump(sentences, f)

with open('./sentences.pkl', 'rb') as f:
  sentences = pickle.load(f)

print(len(sentences))

500000


In [7]:
# Тренируем и сохраняем модель fasttext
sentences = [i for i in sentences if len(i) > 2] 
modelFT = FastText(sentences=sentences, 
                   min_count=1, 
                   window=5, 
                   workers=cpu_count())

In [8]:
modelFT.save("./ft_model")
modelFT = FastText.load("./ft_model")

In [9]:
def prepared_answers_preprocess_txt(lines):
    index_map = {}
    vector_ft_list = []
    counter_list = []
    question_list = []
    for counter, line in enumerate(tqdm_notebook(lines)):
        n_ft = 0
        spls = line.replace('\n', '').split("\t")
        index_map[counter] = spls[1].replace('\n', '')
        question = preprocess_txt(line=spls[0], 
                                  sw=sw, 
                                  morpher=morpher, 
                                  exclude=exclude, 
                                  skip_stop_word=True)
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        
        vector_ft_list.append(vector_ft)
        counter_list.append(counter)
        question_list.append(question)
        
    return {'vector_ft_list': vector_ft_list, 
            'index_map': index_map, 
            'counter_list': counter_list, 
            'question_list': question_list}
    
file_lines = None
with open("prepared_answers.txt", "r") as f:
    file_lines = list(f)

args_parallel = []
for lines in ar_split_eq_cpu(file_lines):
    args_parallel.append(lines)

# запускаем процесс параллельных вычислений
res_raw = paralell_execution(func=prepared_answers_preprocess_txt,
                         arg=args_parallel,
                         method='multiprocessing')


ft_index = annoy.AnnoyIndex(100 ,'angular')
glob_counter = 0
index_map = {}
index_map_questions = {}
for item in res_raw:
    vector_ft_list = item['vector_ft_list']
    index_map_list = item['index_map']
    counter_list = item['counter_list']
    question_list = item['question_list']
    
    for counter in counter_list:
        index_map[glob_counter] = index_map_list[counter]
        index_map_questions[glob_counter] = question_list[counter]
        ft_index.add_item(glob_counter, vector_ft_list[counter])
        glob_counter += 1

ft_index.build(10)
ft_index.save('./speaker.ann')
ft_index.load('./speaker.ann') 

True

In [10]:
with open('index_map.pkl', 'wb') as fp:
   pickle.dump(index_map, fp)

with open('index_map.pkl', 'rb') as f:
  index_map = pickle.load(f)

In [14]:
# vectorizer = CountVectorizer(ngram_range=(1, 2))
# texts = [" ".join(v) for v in index_map_questions.values()]
# vectorizer.fit_transform(texts)

In [15]:
# # Сохраняем результаты
# vectorizer = None
# with open('./vectorizer.pkl', 'wb') as f:
#    pickle.dump(vectorizer, f)

# with open('./vectorizer.pkl', 'rb') as f:
#   vectorizer = pickle.load(f)

In [18]:
params_chat = {
    'sw': sw, 
    'morpher': morpher, 
    'exclude': exclude, 
    'modelFT': modelFT, 
}
with open('./params_chat.pkl', 'wb') as f:
    pickle.dump(params_chat, f)
with open('./params_chat.pkl', 'rb') as f:
    params_chat = pickle.load(f)
    
ft_index = annoy.AnnoyIndex(100 ,'angular')
ft_index.load('./speaker.ann') 
params_chat['ft_index'] = ft_index 

## Интент текущего времени сервера

In [13]:
def cur_time():
    now = datetime.now()     
    return f'Текущее время сервера: {now.strftime("%d-%m-%Y %H:%M:%S")}'

## Интент перевода на английский с помощью нейронной сети из ДЗ 10-го урока

In [2]:
from translator import Encoder, Decoder, preprocess_sentence, evaluate, translate
import pickle
import tensorflow as tf
import os

# Загружаем предобученные модели
inp_lang = None
targ_lang = None

with open('./inp_lang.pickle', "rb") as f:
    inp_lang = pickle.load(f)
with open('./targ_lang.pickle', "rb") as f:
    targ_lang = pickle.load(f)
    
max_length_targ = 12
max_length_inp = 16
BATCH_SIZE = 64
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
optimizer = tf.keras.optimizers.Adam()

# Сохраняем параметры в файл для более быстрой загрузки в файле кода бота
params_translator = {
    'max_length_targ': max_length_targ, 
    'max_length_inp': max_length_inp, 
    'inp_lang': inp_lang, 
    'targ_lang': targ_lang, 
    'units': units, 
    'vocab_inp_size': vocab_inp_size, 
    'vocab_tar_size': vocab_tar_size, 
    'embedding_dim': embedding_dim, 
    'BATCH_SIZE': BATCH_SIZE, 
}
with open('./params_translator.pkl', 'wb') as f:
   pickle.dump(params_translator, f)
with open('./params_translator.pkl', 'rb') as f:
  params_translator = pickle.load(f)

encoder = Encoder(params_translator['vocab_inp_size'], 
                  params_translator['embedding_dim'], 
                  params_translator['units'], 
                  params_translator['BATCH_SIZE'])
decoder = Decoder(params_translator['vocab_tar_size'], 
                  params_translator['embedding_dim'], 
                  params_translator['units'], 
                  params_translator['BATCH_SIZE'])

checkpoint_dir = './training_nmt_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

params_translator['encoder'] = encoder
params_translator['decoder'] = decoder

print( translate('переведи добрый день', params_translator) )
print( translate('переведи хороший выбор', params_translator) )

2023-12-03 16:41:00.825008: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


good afternoon . 
it's a good choice . 


## Функция генерации ответов бота к пользователю

In [1]:
from tg import process
from chat import load_chat_params
chat_params = load_chat_params()

2023-12-03 17:34:54.746008: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [21]:
process('который час', chat_params)

'Текущее время сервера: 03-12-2023 15:33:28'

In [27]:
process('переведи добрый день', chat_params)

'good afternoon . '

In [23]:
process('давай поговорим', chat_params)

'Мы молчим???Нас тут не угомонишь.... '

## Телеграм бот

Бот запускается через файл, в ноутбуке он не работает.

python3 tg.py

Результат работы бота:

![](result.jpg)